In [2]:
import cv2
import numpy as np
import csv
import os

In [3]:
#folder data
base_folder = "data"
subfolders = ["A", "M"]

In [8]:
#menyiapkan file csv untuk menyimpan fitur
with open('geometric_feature.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Label', 'Area', 'Parameter', 'Aspect Ratio', 'Solidity', 'Eccentricity'])
    
    #iterasi melalui setiap folder dan gambar
    for folder in subfolders:
        folder_path = os.path.join(base_folder, folder)
        
        for image_name in os.listdir(folder_path):
            image_path = os.path.join(folder_path, image_name)
            
            #membaca gambar dalam format BGR
            image = cv2.imread(image_path)
            if image is None:
                continue
            
            #konversi ke grayscale
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
                
            #pra pemrosesan gambar
            blurred = cv2.GaussianBlur(gray, (5, 5), 0)
            _, thresh = cv2.threshold(blurred, 127, 255, cv2.THRESH_BINARY)
            
            #mendeteksi kontur
            contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            if not contours or len(contours) == 0:
                continue
                
            largest_contour = max(contours, key=cv2.contourArea)
            
            # Pastikan kontur memiliki cukup titik untuk fitting ellipse
            if len(largest_contour) < 5:
                continue
            
            #ekstraksi fitur geometri
            area = cv2.contourArea(largest_contour)
            perimeter = cv2.arcLength(largest_contour, True)
            hull = cv2.convexHull(largest_contour)
            x, y, w, h = cv2.boundingRect(largest_contour)
            aspect_ratio = float(w) / h if h != 0 else 0
            hull_area = cv2.contourArea(hull)
            solidity = float(area) / hull_area if hull_area != 0 else 0
            
            try:
                (x, y), (MA, ma), angle = cv2.fitEllipse(largest_contour)
                # Pastikan MA (major axis) lebih besar dari ma (minor axis)
                if MA > ma and MA != 0:
                    # Rumus eccentricity yang benar untuk ellipse
                    eccentricity = np.sqrt(1 - (ma/MA)**2)
                else:
                    eccentricity = 0
            except:
                eccentricity = 0
            
            #menambahkan label berdasarkan folder
            label = folder
            features = [label, area, perimeter, aspect_ratio, solidity, eccentricity]
            writer.writerow(features)
            
print("Ekstraksi Fitur Geometri sudah selesai dan disimpan di file 'geometric_feature.csv'")

Ekstraksi Fitur Geometri sudah selesai dan disimpan di file 'geometric_feature.csv'
